In [1]:
from graph_tool import Graph, VertexPropertyMap, util, draw, inference
import polars as pl
import math

## Initialisation & Exploration


In [2]:
g: Graph = Graph(directed=True)

In [3]:
df_users: pl.DataFrame = pl.read_parquet("../collection/users.parquet")
df_questions: pl.DataFrame = pl.read_parquet("../collection/questions.parquet")
df_answers: pl.DataFrame = pl.read_parquet("../collection/answers.parquet")

In [4]:
(shape_users, shape_questions, shape_answers) = (
    df_users.shape,
    df_questions.shape,
    df_answers.shape,
)
print(f"Users: {shape_users}, Questions: {shape_questions}, Answers: {shape_answers}")
print(
    f"Total number of nodes: {shape_users[0] + shape_questions[0] + shape_answers[0]}"
)

Users: (1893, 28), Questions: (263, 39), Answers: (1909, 26)
Total number of nodes: 4065


In [5]:
df_users.head()

shape: (5, 28)
┌────────────┬────────────┬────────────┬──────┬───┬─────────┬────────────┬────────────┬────────────┐
│ about_me   ┆ accept_rat ┆ account_id ┆ age  ┆ … ┆ user_id ┆ user_type  ┆ view_count ┆ website_ur │
│ ---        ┆ e          ┆ ---        ┆ ---  ┆   ┆ ---     ┆ ---        ┆ ---        ┆ l          │
│ str        ┆ ---        ┆ i64        ┆ i64  ┆   ┆ i64     ┆ cat        ┆ i64        ┆ ---        │
│            ┆ i64        ┆            ┆      ┆   ┆         ┆            ┆            ┆ str        │
╞════════════╪════════════╪════════════╪══════╪═══╪═════════╪════════════╪════════════╪════════════╡
│ <p>Author  ┆ 86         ┆ 11683      ┆ null ┆ … ┆ 22656   ┆ registered ┆ 2283688    ┆ http://csh │
│ of <a href ┆            ┆            ┆      ┆   ┆         ┆            ┆            ┆ arpindepth │
│ ="https:// ┆            ┆            ┆      ┆   ┆         ┆            ┆            ┆ .com       │
│ ww…        ┆            ┆            ┆      ┆   ┆         ┆            ┆            ┆            │
│ <p>Author  ┆ 86         ┆ 11683      ┆ null ┆ … ┆ 22656   ┆ registered ┆ 2283688    ┆ http://csh │
│ of <a href ┆            ┆            ┆      ┆   ┆         ┆            ┆            ┆ arpindepth │
│ ="https:// ┆            ┆            ┆      ┆   ┆         ┆            ┆            ┆ .com       │
│ ww…        ┆            ┆            ┆      ┆   ┆         ┆            ┆            ┆            │
│ <p>You     ┆ 94         ┆ 335069     ┆ null ┆ … ┆ 661933  ┆ registered ┆ 14536      ┆            │
│ should     ┆            ┆            ┆      ┆   ┆         ┆            ┆            ┆            │
│ accept the ┆            ┆            ┆      ┆   ┆         ┆            ┆            ┆            │
│ answer …   ┆            ┆            ┆      ┆   ┆         ┆            ┆            ┆            │
│ <p>I write ┆ 50         ┆ 3990       ┆ null ┆ … ┆ 5862    ┆ registered ┆ 996        ┆ http://sil │
│ code.</p>  ┆            ┆            ┆      ┆   ┆         ┆            ┆            ┆ iconsea.ne │
│            ┆            ┆            ┆      ┆   ┆         ┆            ┆            ┆ t/         │
│ <p>Develop ┆ 50         ┆ 147914     ┆ null ┆ … ┆ 359349  ┆ registered ┆ 200        ┆            │
│ er with 8+ ┆            ┆            ┆      ┆   ┆         ┆            ┆            ┆            │
│ years      ┆            ┆            ┆      ┆   ┆         ┆            ┆            ┆            │
│ exper…     ┆            ┆            ┆      ┆   ┆         ┆            ┆            ┆            │
└────────────┴────────────┴────────────┴──────┴───┴─────────┴────────────┴────────────┴────────────┘

From a user we can identity the following interesting information:

- The user's Account ID: `account_id`
  - This is very important because it is the unique identifier for the user ACCROSS the entire Stack Exchange network.
- The user's ID: `user_id`
  - This is the unique identifier for the user on the specific site (here, Stack Overflow).
- The number of answers the user has given: `answer_count`
- The number of questions the user has asked: `question_count`
- The number of upvotes the user has received: `up_vote_count`
- The number of downvotes the user has received: `down_vote_count`
- The rate of the user's answers being accepted: `accept_rate`
- The user's location: `location`
- The users's reputation: `reputation`


In [6]:
df_questions.head()

shape: (5, 39)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ accepted_ ┆ answer_co ┆ answers   ┆ body      ┆ … ┆ tags      ┆ title     ┆ up_vote_c ┆ view_cou │
│ answer_id ┆ unt       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ount      ┆ nt       │
│ ---       ┆ ---       ┆ list[stru ┆ str       ┆   ┆ list[str] ┆ str       ┆ ---       ┆ ---      │
│ i64       ┆ i64       ┆ ct[26]]   ┆           ┆   ┆           ┆           ┆ i64       ┆ i64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 247623    ┆ 13        ┆ [{247623, ┆ <p>What   ┆ … ┆ ["c#",    ┆ What are  ┆ 2917      ┆ 423801   │
│           ┆           ┆ null,null ┆ are the   ┆   ┆ ".net", … ┆ the       ┆           ┆          │
│           ┆           ┆ ,null,nul ┆ correct   ┆   ┆ ".net-fra ┆ correct   ┆           ┆          │
│           ┆           ┆ l,nul…    ┆ version … ┆   ┆ mework…   ┆ version   ┆           ┆          │
│           ┆           ┆           ┆           ┆   ┆           ┆ num…      ┆           ┆          │
│ 77423144  ┆ 2         ┆ [{7742301 ┆ <p>I have ┆ … ┆ ["wpf"]   ┆ How can I ┆ 2         ┆ 280      │
│           ┆           ┆ 7,null,nu ┆ a <code>S ┆   ┆           ┆ only      ┆           ┆          │
│           ┆           ┆ ll,null,n ┆ crollView ┆   ┆           ┆ scroll a  ┆           ┆          │
│           ┆           ┆ ull,n…    ┆ er</…     ┆   ┆           ┆ single n… ┆           ┆          │
│ 76172541  ┆ 2         ┆ [{7617254 ┆ <p>I      ┆ … ┆ ["string" ┆ How can I ┆ 4         ┆ 638      │
│           ┆           ┆ 1,null,nu ┆ receive a ┆   ┆ ,         ┆ trim      ┆           ┆          │
│           ┆           ┆ ll,null,n ┆ string    ┆   ┆ "vb.net", ┆ double    ┆           ┆          │
│           ┆           ┆ ull,n…    ┆ which     ┆   ┆ "trim"]   ┆ quotes    ┆           ┆          │
│           ┆           ┆           ┆ some…     ┆   ┆           ┆ fro…      ┆           ┆          │
│ 74327430  ┆ 2         ┆ [{7432743 ┆ <p>I'm    ┆ … ┆ ["c#",    ┆ Why does  ┆ 107       ┆ 6958     │
│           ┆           ┆ 0,null,nu ┆ trying to ┆   ┆ "udp",    ┆ sending   ┆           ┆          │
│           ┆           ┆ ll,null,n ┆ create a  ┆   ┆ "udpclien ┆ via a Udp ┆           ┆          │
│           ┆           ┆ ull,n…    ┆ UDP se…   ┆   ┆ t"]       ┆ Client…   ┆           ┆          │
│ null      ┆ 407       ┆ [{406762, ┆ <p>This   ┆ … ┆ ["languag ┆ What's    ┆ 386       ┆ 308457   │
│           ┆           ┆ null,null ┆ is defini ┆   ┆ e-agnosti ┆ your most ┆           ┆          │
│           ┆           ┆ ,null,nul ┆ tely subj ┆   ┆ c"]       ┆ controver ┆           ┆          │
│           ┆           ┆ l,nul…    ┆ ective…   ┆   ┆           ┆ sial p…   ┆           ┆          │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴──────────┘

From a question we can identity the following interesting information:

- The question's ID: `question_id`
- The user who asked the question: `owner`
  - Note that this is a "shallow user" object, from which we can extract the `user_id` of the user.
- The accepted answer's ID to the question: `accepted_answer_id` (if it exists)
- The number of answers the question has received: `answer_count`
- The number of upvotes the question has received: `up_vote_count`
- The number of downvotes the question has received: `down_vote_count`
- The number of favorites the question has received: `favorite_count`
- The last editor of the question: `last_editor`
  - Note that this is a "shallow user" object, from which we can extract the `user_id` of the user.
- The number of views the question has received: `view_count`
- The question's tags: `tags`
  - Note that this is a list of strings, which can be used to identify the topics of the question.
  - We could potentially link questions with similar tags together.


In [7]:
df_answers.head()

shape: (5, 26)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ answer_id ┆ awarded_b ┆ awarded_b ┆ body      ┆ … ┆ share_lin ┆ tags      ┆ title     ┆ up_vote_ │
│ ---       ┆ ounty_amo ┆ ounty_use ┆ ---       ┆   ┆ k         ┆ ---       ┆ ---       ┆ count    │
│ i64       ┆ unt       ┆ rs        ┆ str       ┆   ┆ ---       ┆ list[str] ┆ str       ┆ ---      │
│           ┆ ---       ┆ ---       ┆           ┆   ┆ str       ┆           ┆           ┆ i64      │
│           ┆ i64       ┆ list[stru ┆           ┆   ┆           ┆           ┆           ┆          │
│           ┆           ┆ ct[8]]    ┆           ┆   ┆           ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 247623    ┆ null      ┆ null      ┆ <h1>C#    ┆ … ┆ https://s ┆ []        ┆ What are  ┆ 3193     │
│           ┆           ┆           ┆ language  ┆   ┆ tackoverf ┆           ┆ the       ┆          │
│           ┆           ┆           ┆ version   ┆   ┆ low.com/a ┆           ┆ correct   ┆          │
│           ┆           ┆           ┆ history:… ┆   ┆ /2476…    ┆           ┆ version   ┆          │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆ num…      ┆          │
│ 38506668  ┆ null      ┆ null      ┆ <p>This   ┆ … ┆ https://s ┆ []        ┆ What are  ┆ 481      │
│           ┆           ┆           ┆ is the    ┆   ┆ tackoverf ┆           ┆ the       ┆          │
│           ┆           ┆           ┆ same as   ┆   ┆ low.com/a ┆           ┆ correct   ┆          │
│           ┆           ┆           ┆ most      ┆   ┆ /3850…    ┆           ┆ version   ┆          │
│           ┆           ┆           ┆ answ…     ┆   ┆           ┆           ┆ num…      ┆          │
│ 317885    ┆ null      ┆ null      ┆ <p>The    ┆ … ┆ https://s ┆ []        ┆ What are  ┆ 330      │
│           ┆           ┆           ┆ biggest   ┆   ┆ tackoverf ┆           ┆ the       ┆          │
│           ┆           ┆           ┆ problem   ┆   ┆ low.com/a ┆           ┆ correct   ┆          │
│           ┆           ┆           ┆ when      ┆   ┆ /3178…    ┆           ┆ version   ┆          │
│           ┆           ┆           ┆ deal…     ┆   ┆           ┆           ┆ num…      ┆          │
│ 4570323   ┆ null      ┆ null      ┆ <ul>      ┆ … ┆ https://s ┆ []        ┆ What are  ┆ 225      │
│           ┆           ┆           ┆ <li><p><s ┆   ┆ tackoverf ┆           ┆ the       ┆          │
│           ┆           ┆           ┆ trong>C#  ┆   ┆ low.com/a ┆           ┆ correct   ┆          │
│           ┆           ┆           ┆ 1.0</stro ┆   ┆ /4570…    ┆           ┆ version   ┆          │
│           ┆           ┆           ┆ …         ┆   ┆           ┆           ┆ num…      ┆          │
│ 2788024   ┆ null      ┆ null      ┆ <div clas ┆ … ┆ https://s ┆ []        ┆ What are  ┆ 103      │
│           ┆           ┆           ┆ s="s-tabl ┆   ┆ tackoverf ┆           ┆ the       ┆          │
│           ┆           ┆           ┆ e-contain ┆   ┆ low.com/a ┆           ┆ correct   ┆          │
│           ┆           ┆           ┆ er">      ┆   ┆ /2788…    ┆           ┆ version   ┆          │
│           ┆           ┆           ┆ …         ┆   ┆           ┆           ┆ num…      ┆          │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴──────────┘

From an answer we can identity the following interesting information:

- The answer's ID: `answer_id`
- The user who answered the question: `owner`
  - Note that this is a "shallow user" object, from which we can extract the `user_id` of the user.
- The question the answer is answering: `question_id`
- The number of upvotes the answer has received: `up_vote_count`
- The number of downvotes the answer has received: `down_vote_count`
- The number of comments the answer has received: `comment_count`
- The answer's score: `score`
- The answer's tags: `tags`
  - Note that this is a list of strings, which can be used to identify the topics of the answer.
  - We could potentially link answers with similar tags together.
- The last editor of the answer: `last_editor`
  - Note that this is a "shallow user" object, from which we can extract the `user_id` of the user.


## Data cleaning


We are going to to drop the columns that we do not want to use in our analysis. We are also going to drop the rows that have missing values.


In [8]:
df_users_cols: list[str] = df_users.columns
df_users_cols_to_keep: list[str] = [
    "user_id",
    "answer_count",
    "question_count",
    "up_vote_count",
    "down_vote_count",
    "accept_rate",
    "location",
    "reputation",
]
df_users_cols_to_drop: list[str] = list(set(df_users_cols) - set(df_users_cols_to_keep))
df_users: pl.DataFrame = df_users.drop(df_users_cols_to_drop)
df_users: pl.DataFrame = df_users.drop_nulls()

In [9]:
df_questions_cols: list[str] = df_questions.columns
df_questions_cols_to_keep: list[str] = [
    "question_id",
    "owner",
    "accepted_answer_id",
    "answer_count",
    "up_vote_count",
    "down_vote_count",
    "favorite_count",
    "last_editor",
    "view_count",
    "tags",
]
df_questions_cols_to_drop: list[str] = list(
    set(df_questions_cols) - set(df_questions_cols_to_keep)
)
df_questions: pl.DataFrame = df_questions.drop(df_questions_cols_to_drop)

# Some columns may contain null values, so we need to drop them
# but it's normal for some columns to contain null values
df_questions_cols_to_consider: list[str] = list(
    set(df_questions_cols_to_keep)
    - {
        "accepted_answer_id",
        "last_editor",
    }
)
df_questions: pl.DataFrame = df_questions.drop_nulls(
    subset=df_questions_cols_to_consider
)

In [10]:
df_answers_cols: list[str] = df_answers.columns
df_answers_cols_to_keep: list[str] = [
    "answer_id",
    "owner",
    "question_id",
    "up_vote_count",
    "down_vote_count",
    "comment_count",
    "score",
    "tags",
    "last_editor",
]
df_answers_cols_to_drop: list[str] = list(
    set(df_answers_cols) - set(df_answers_cols_to_keep)
)
df_answers: pl.DataFrame = df_answers.drop(df_answers_cols_to_drop)

# Some columns may contain null values, so we need to drop them
# but it's normal for some columns to contain null values
df_answers_cols_to_consider: list[str] = list(
    set(df_answers_cols_to_keep) - {"last_editor"}
)
df_answers: pl.DataFrame = df_answers.drop_nulls(subset=df_answers_cols_to_consider)

In [11]:
# Remove duplicate rows (same id)
df_users: pl.DataFrame = df_users.unique(subset=["user_id"])
df_questions: pl.DataFrame = df_questions.unique(subset=["question_id"])
df_answers: pl.DataFrame = df_answers.unique(subset=["answer_id"])

## Creation of the Graph


### Create the properties


In [12]:
user_vertex_color: str = "#FED869"
question_vertex_color: str = "#ED6B89"
answer_vertex_color: str = "#91D9E6"

v_prop_color: VertexPropertyMap = g.new_vertex_property("string")
v_prop_type = g.new_vertex_property("string")

In [13]:
u_prop_user_id: VertexPropertyMap = g.new_vertex_property("string")
u_prop_answer_count: VertexPropertyMap = g.new_vertex_property("int")
u_prop_question_count: VertexPropertyMap = g.new_vertex_property("int")
u_prop_up_vote_count: VertexPropertyMap = g.new_vertex_property("int")
u_prop_down_vote_count: VertexPropertyMap = g.new_vertex_property("int")
u_prop_accept_rate: VertexPropertyMap = g.new_vertex_property("float")
u_prop_location: VertexPropertyMap = g.new_vertex_property("string")
u_prop_reputation: VertexPropertyMap = g.new_vertex_property("int")

q_prop_question_id: VertexPropertyMap = g.new_vertex_property("string")
q_prop_owner_id: VertexPropertyMap = g.new_vertex_property("string")
q_prop_accepted_answer_id: VertexPropertyMap = g.new_vertex_property("string")
q_prop_answer_count: VertexPropertyMap = g.new_vertex_property("int")
q_prop_up_vote_count: VertexPropertyMap = g.new_vertex_property("int")
q_prop_down_vote_count: VertexPropertyMap = g.new_vertex_property("int")
q_prop_favorite_count: VertexPropertyMap = g.new_vertex_property("int")
q_prop_last_editor_id: VertexPropertyMap = g.new_vertex_property("string")
q_prop_view_count: VertexPropertyMap = g.new_vertex_property("int")
q_prop_tags: VertexPropertyMap = g.new_vertex_property("vector<string>")

a_prop_answer_id: VertexPropertyMap = g.new_vertex_property("string")
a_prop_owner_id: VertexPropertyMap = g.new_vertex_property("string")
a_prop_question_id: VertexPropertyMap = g.new_vertex_property("string")
a_prop_up_vote_count: VertexPropertyMap = g.new_vertex_property("int")
a_prop_down_vote_count: VertexPropertyMap = g.new_vertex_property("int")
a_prop_comment_count: VertexPropertyMap = g.new_vertex_property("int")
a_prop_score: VertexPropertyMap = g.new_vertex_property("int")
a_prop_tags: VertexPropertyMap = g.new_vertex_property("vector<string>")
a_prop_last_editor_id: VertexPropertyMap = g.new_vertex_property("string")

### Create the nodes


In [14]:
users_dict = df_users.to_dict()

for i, user in enumerate(users_dict["user_id"]):
    u = g.add_vertex()
    v_prop_color[u] = user_vertex_color
    v_prop_type[u] = "user"
    u_prop_user_id[u] = user
    u_prop_answer_count[u] = users_dict["answer_count"][i] or 0
    u_prop_question_count[u] = users_dict["question_count"][i] or 0
    u_prop_up_vote_count[u] = users_dict["up_vote_count"][i] or 0
    u_prop_down_vote_count[u] = users_dict["down_vote_count"][i] or 0
    u_prop_accept_rate[u] = users_dict["accept_rate"][i] or 0
    u_prop_location[u] = users_dict["location"][i] or "Unknown"
    u_prop_reputation[u] = users_dict["reputation"][i] or 0

questions_dict = df_questions.to_dict()

for i, question in enumerate(questions_dict["question_id"]):
    q = g.add_vertex()
    v_prop_color[q] = question_vertex_color
    v_prop_type[q] = "question"
    q_prop_question_id[q] = question
    q_prop_owner_id[q] = questions_dict["owner"][i]["user_id"]
    q_prop_accepted_answer_id[q] = questions_dict["accepted_answer_id"][i] or -1
    q_prop_answer_count[q] = questions_dict["answer_count"][i] or 0
    q_prop_up_vote_count[q] = questions_dict["up_vote_count"][i] or 0
    q_prop_down_vote_count[q] = questions_dict["down_vote_count"][i] or 0
    q_prop_favorite_count[q] = questions_dict["favorite_count"][i] or 0
    q_prop_last_editor_id[q] = questions_dict["last_editor"][i]["user_id"] or "Unknown"
    q_prop_view_count[q] = questions_dict["view_count"][i] or 0
    q_prop_tags[q] = questions_dict["tags"][i]

answers_dict = df_answers.to_dict()

for i, answer in enumerate(answers_dict["answer_id"]):
    a = g.add_vertex()
    v_prop_color[a] = answer_vertex_color
    v_prop_type[a] = "answer"
    a_prop_answer_id[a] = answer
    a_prop_owner_id[a] = answers_dict["owner"][i]["user_id"]
    a_prop_question_id[a] = answers_dict["question_id"][i]
    a_prop_up_vote_count[a] = answers_dict["up_vote_count"][i] or 0
    a_prop_down_vote_count[a] = answers_dict["down_vote_count"][i] or 0
    a_prop_comment_count[a] = answers_dict["comment_count"][i] or 0
    a_prop_score[a] = answers_dict["score"][i] or 0
    a_prop_tags[a] = answers_dict["tags"][i]
    a_prop_last_editor_id[a] = answers_dict["last_editor"][i]["user_id"] or "Unknown"

### Create the relationships


In [15]:
edge_prop_text = g.new_edge_property("string")

In [16]:
for i, question in enumerate(questions_dict["question_id"]):
    # Find the vertex in the graph that corresponds to the current question ID
    matches = util.find_vertex(g, q_prop_question_id, question)

    # If no matching vertex is found, print a message and skip to the next iteration
    if len(matches) == 0:
        print(f"Question with id {question} not found")
        continue

    # Assuming a match is found, get the first matching vertex (should be unique)
    q = matches[0]

    # Check if the question has an accepted answer
    maybe_answer_id = q_prop_accepted_answer_id[q]
    if maybe_answer_id is not None and len(maybe_answer_id) > 0:
        # Find the vertex for the accepted answer
        matches = util.find_vertex(g, a_prop_answer_id, maybe_answer_id)
        if len(matches) == 0:
            # If no matching answer is found and it's not a placeholder ID, print a message
            if maybe_answer_id not in [-1, "-1"]:
                print(f"Answer with id {maybe_answer_id} not found")
            continue
        # Ensure the found vertex is of type "answer"
        if v_prop_type[matches[0]] != "answer":
            print(f"Vertex with id {maybe_answer_id} is not an answer")
            continue
        # Create an edge from the answer to the question
        a = matches[0]
        e = g.add_edge(a, q)
        edge_prop_text[e] = "Answer to"

    # Find the user vertex who asked the question
    maybe_owner = util.find_vertex(g, u_prop_user_id, q_prop_owner_id[q])
    if (
        maybe_owner is not None
        and len(maybe_owner) > 0
        and v_prop_type[maybe_owner[0]] == "user"
    ):
        # Create an edge from the user to the question
        u = maybe_owner[0]
        e = g.add_edge(u, q)
        edge_prop_text[e] = "Asked"

# Repeat a similar process for answers
for i, answer in enumerate(answers_dict["answer_id"]):
    # Find the vertex for the current answer ID
    matches = util.find_vertex(g, a_prop_answer_id, answer)

    # If no matching vertex is found, print a message and skip to the next iteration
    if len(matches) == 0:
        if answer not in [-1, "-1"]:
            print(f"Answer with id {answer} not found")
        continue
    # Ensure the found vertex is of type "answer"
    if v_prop_type[matches[0]] != "answer":
        print(f"Vertex with id {answer} is not an answer")
        continue

    a = matches[0]

    # Check if the answer is linked to a question
    maybe_question_id = a_prop_question_id[a]
    if maybe_question_id is not None and len(maybe_question_id) > 0:
        # Find the question vertex
        matches = util.find_vertex(g, q_prop_question_id, maybe_question_id)
        if len(matches) == 0:
            if maybe_question_id != -1:
                print(f"Question with id {maybe_question_id} not found")
            continue
        # Ensure the found vertex is of type "question"
        if v_prop_type[matches[0]] != "question":
            print(f"Vertex with id {maybe_question_id} is not a question")
            continue
        # Create an edge from the question to the answer
        q = matches[0]
        e = g.add_edge(q, a)
        edge_prop_text[e] = "Answer"

    # Find the user vertex who answered
    maybe_owner = util.find_vertex(g, u_prop_user_id, a_prop_owner_id[a])
    if (
        maybe_owner is not None
        and len(maybe_owner) > 0
        and v_prop_type[maybe_owner[0]] == "user"
    ):
        # Create an edge from the user to the answer
        u = maybe_owner[0]
        e = g.add_edge(u, a)
        edge_prop_text[e] = "Answered"

## Visualisation


In [17]:
num_v = g.num_vertices()
num_e = g.num_edges()
print(f"Number of vertices: {num_v}")
print(f"Number of edges: {num_e}")

print(
    f"Obtaining the SFDP spring-block layout of the graph. The complexity of this layout is O(V * log(V)) = {num_v} * log({num_v}) = {num_v * math.log(num_v)}"
)
pos_sfdp = draw.sfdp_layout(g)
print(
    f"Obtaining the radial layout of the graph. The complexity of this layout is O(V + E) = {num_v} + {num_e} = {num_v + num_e}"
)
pos_radial_tree = draw.radial_tree_layout(g, g.vertex(0))

Number of vertices: 2954
Number of edges: 3227
Obtaining the SFDP spring-block layout of the graph. The complexity of this layout is O(V * log(V)) = 2954 * log(2954) = 23605.164277971777
Obtaining the radial layout of the graph. The complexity of this layout is O(V + E) = 2954 + 3227 = 6181


In [18]:
draw.graph_draw(
    g,
    pos=pos_sfdp,
    vertex_fill_color=v_prop_color,
    vertex_text=u_prop_user_id,
    edge_text=edge_prop_text,
    output_size=(5000, 5000),
    output="sfdp_graph.png",
    ink_scale=0.3,  # type: ignore
)

draw.graph_draw(
    g,
    pos=pos_radial_tree,
    vertex_fill_color=v_prop_color,
    vertex_text=u_prop_user_id,
    edge_text=edge_prop_text,
    output_size=(5000, 5000),
    output="radial_graph.png",
    ink_scale=0.3,  # type: ignore
)

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x16db786e0, at 0x1064c56d0>